In [ ]:
# pip uninstall tensorflow
# !pip3 install -q geopandas xarray GitPython
# !conda install -y -c conda-forge cartopy
# !pip install shapely --upgrade
# !pip install cartopy --upgrade
# !pip install netCDF4

In [ ]:
from pathlib import Path

import GNN_Arbitrary
from git import Repo
import json

from Settings.Settings import *

class AttrDict(dict):
    def __getattr__(self, key):
        return self[key]

    def __setattr__(self, key, value):
        self[key] = value

In [ ]:
for era_5_nodes in [0,4]:
    for lear_hours in [1,4,8,16,24,48,96]:

        args = AttrDict()

        model_name = 'NoArbitrary'
        repo = Repo(".")
        commitHash = repo.git.rev_parse("HEAD")

        # args.p_arbitrary_nodes = .05
        args.lead_hrs = lear_hours
        args.n_neighbors_e2m = era_5_nodes
        # args.station_sampling_method = StationSamplingMethod.random_shared
        # args.temporal_dataset_split = TemporalDatasetSplit.none


        # args.output_saving_path = f'ModelOutputs/{model_name}_{commitHash}_' + ''.join([f'_{k}={v.name if issubclass(type(v), Enum) else v}' for k, v in args.items()])
        args.output_saving_path = f'ModelOutputs/{model_name}_' + ''.join([f'_{k}={v.name if issubclass(type(v), Enum) else v}' for k, v in args.items()])


        ######### args after this are not included in path name

        args.static_node_type = StaticNodeType.none
        args.temporal_dataset_split = TemporalDatasetSplit.holdout_fixed
        args.station_sampling_method = StationSamplingMethod.none
        args.p_arbitrary_nodes = 0
        args.static_node_image_size = 32

        args.network_construction_method = 'KNN' #NetworkConstructionMethod.KNN
        args.p_arbitrary_nodes_valid = .2
        args.p_arbitrary_nodes_test = .1
        args.loss_factor = 1

        args.coords = (-74, -70, 41, 43)
        args.back_hrs = 24
        #args.ERA5_len = 25
        args.hidden_dim = 128
        args.lr = 1e-4
        args.epochs = 60
        args.batch_size = 128
        args.weight_decay = 1e-4
        args.model_type = 'GNN'
        #args.shuffle_data = False
        args.n_years = 5
        args.n_passing = 4
        args.n_neighbors_m2m = 5
        args.edge_type = 'None'

        save_args = args.copy()

        for k in save_args.keys():
            v = save_args[k]
            if issubclass(type(v), Enum):
                save_args[k] = v.name

        args.eval_interval = 5
        args.root_path = '/Users/jonathangiezendanner/Documents/MIT/Projects/WindData/'
        args.show_progress_bar = True

        outputPath = (Path(args.root_path)/args.output_saving_path/'params.json')

        outputPath.parent.mkdir(exist_ok=True, parents=True)

        with open(outputPath, 'w') as f:
            json.dump(save_args, f)

        GNN_Arbitrary.Run(args)
